In [6]:
import cv2
import numpy as np
import tensorflow as tf

# Load model TFLite
interpreter = tf.lite.Interpreter(model_path="best_model_v2.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Contoh labels (26 kelas), sesuaikan dengan label asli kamu
labels = [
    "A", "B", "C", "D", "E",
    "F", "G", "H", "I", "J",
    "K", "L", "M", "N", "O",
    "P", "Q", "R", "S", "T",
    "U", "V", "W", "X", "Y",
    "Z"
]

# Fungsi preprocess frame
def preprocess_frame(frame):
    # Resize sesuai input model (biasanya 224x224)
    img = cv2.resize(frame, (224, 224))
    # Ubah ke float32 dan normalisasi jika perlu
    img = img.astype(np.float32) / 255.0
    # Tambah batch dimension
    img = np.expand_dims(img, axis=0)
    return img

# Buka kamera
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess frame
    input_data = preprocess_frame(frame)

    # Set input tensor
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()

    # Ambil hasil prediksi
    output_data = interpreter.get_tensor(output_details[0]['index'])
    preds = output_data[0]  # Karena batch=1

    # Cari kelas dengan confidence tertinggi
    top_class = np.argmax(preds)
    confidence = preds[top_class]

    # Pastikan indeks aman
    if top_class < len(labels):
        text = f"{labels[top_class]}: {confidence*100:.2f}%"
    else:
        text = "Unknown"

    # Tampilkan teks di frame
    cv2.putText(frame, text, (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("TFLite Camera Prediction", frame)

    # Tekan 'q' untuk keluarq
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()